In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
!pip install nbconvert==6.0.0
import nbconvert

In [ ]:
#loading data
key = os.environ.get('AZURE_SEARCH_KEY', 'xxxxxxxxxxxxx')

In [ ]:
search_images_bing

In [ ]:
results = search_images_bing(key, 'seal')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
pinnipeds = 'earless seal', 'sea lion', 'fur seal'
path = Path('pinnipeds')

In [ ]:
if not path.exists():
    path.mkdir()
    for p in pinnipeds:
        dest = (path/p)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{p}')
        download_images(dest, urls=results.attrgot('content_url'))

In [ ]:
fns=get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
aquatic_mammals = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed = 51),
    get_y = parent_label,
    item_tfms = Resize(128)
    )

In [ ]:
dls = aquatic_mammals.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
aquatic_mammals = aquatic_mammals.new(item_tfms = RandomResizedCrop(128, min_scale = 0.3))
dls = aquatic_mammals.dataloaders(path)
dls.train.show_batch(max_n = 4, nrows = 1, unique = True)

In [ ]:
aquatic_mammals = aquatic_mammals.new(item_tfms = Resize(128), batch_tfms = aug_transforms(mult = 2))
dls = aquatic_mammals.dataloaders(path)
dls.train.show_batch(max_n = 8, nrows = 2, unique = True)

In [ ]:
aquatic_mammals = aquatic_mammals.new(
    item_tfms = RandomResizedCrop(224, min_scale = 0.5),
    batch_tfms = aug_transforms())

dls = aquatic_mammals.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(7, nrows=2)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)